In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
DATAPATH = './data/games.csv'

In [3]:
games = pd.read_csv(DATAPATH)

In [4]:
games.dtypes

Unnamed: 0             int64
SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [5]:
games = games.drop(['Unnamed: 0'], axis=1)

In [6]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612748,MIA,Miami Heat,21900970,2020-03-11,MIA vs. CHA,L,239,98,...,0.833,9,27,36,32,8,5,14,17,-13.0
1,22019,1610612743,DEN,Denver Nuggets,21900973,2020-03-11,DEN @ DAL,L,240,97,...,0.625,3,37,40,23,6,4,15,20,-7.8
2,22019,1610612752,NYK,New York Knicks,21900969,2020-03-11,NYK @ ATL,W,264,136,...,0.735,6,41,47,32,14,9,12,23,0.0
3,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,W,240,113,...,0.739,13,39,52,23,9,2,10,14,8.0
4,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,L,241,106,...,0.680,6,26,32,25,10,3,8,24,-14.2


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('WL')
#     ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [8]:
games.pop('SEASON_ID')
games.pop('TEAM_ID')
games.pop('GAME_ID')
games.pop('TEAM_ABBREVIATION')

0        MIA
1        DEN
2        NYK
3        DAL
4        DET
        ... 
29995    BOS
29996    LAL
29997    ATL
29998    DAL
29999    DEN
Name: TEAM_ABBREVIATION, Length: 30000, dtype: object

In [9]:
feature_columns = []
obj_columns = ['TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL']
#numeric columns
for i in obj_columns:
    games[i] = pd.Categorical(games[i])
    games[i] = games[i].cat.codes 
for header in games.columns:
    games[header] = games[header].astype('float64')
    if header != 'WL':
        feature_columns.append(feature_column.numeric_column(header))

In [10]:
games.dtypes

TEAM_NAME     float64
GAME_DATE     float64
MATCHUP       float64
WL            float64
MIN           float64
PTS           float64
FGM           float64
FGA           float64
FG_PCT        float64
FG3M          float64
FG3A          float64
FG3_PCT       float64
FTM           float64
FTA           float64
FT_PCT        float64
OREB          float64
DREB          float64
REB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PLUS_MINUS    float64
dtype: object

In [11]:
games.head(n=250)

,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42.0,2427.0,1050.0,0.0,239.0,98.0,39.0,84.0,0.464,15.0,...,0.833,9.0,27.0,36.0,32.0,8.0,5.0,14.0,17.0,-13.0
1,22.0,2427.0,465.0,0.0,240.0,97.0,39.0,83.0,0.470,14.0,...,0.625,3.0,37.0,40.0,23.0,6.0,4.0,15.0,20.0,-7.8
2,52.0,2427.0,1395.0,1.0,264.0,136.0,50.0,94.0,0.532,11.0,...,0.735,6.0,41.0,47.0,32.0,14.0,9.0,12.0,23.0,0.0
3,21.0,2427.0,434.0,1.0,240.0,113.0,42.0,92.0,0.457,12.0,...,0.739,13.0,39.0,52.0,23.0,9.0,2.0,10.0,14.0,8.0
4,23.0,2427.0,546.0,0.0,241.0,106.0,39.0,84.0,0.464,11.0,...,0.680,6.0,26.0,32.0,25.0,10.0,3.0,8.0,24.0,-14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,47.0,2411.0,1147.0,0.0,239.0,123.0,42.0,95.0,0.442,15.0,...,0.706,7.0,35.0,42.0,16.0,4.0,6.0,11.0,22.0,-16.0
246,41.0,2411.0,961.0,0.0,241.0,97.0,37.0,89.0,0.416,7.0,...,0.727,11.0,34.0,45.0,20.0,6.0,7.0,13.0,18.0,-27.0
247,20.0,2411.0,371.0,1.0,265.0,125.0,43.0,97.0,0.443,14.0,...,0.735,13.0,40.0,53.0,25.0,8.0,6.0,12.0,20.0,6.0
248,51.0,2410.0,1346.0,1.0,239.0,115.0,46.0,98.0,0.469,17.0,...,0.462,14.0,38.0,52.0,34.0,7.0,5.0,13.0,15.0,14.0


In [12]:
games = games.dropna()

In [13]:
print(feature_columns)

[NumericColumn(key='TEAM_NAME', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GAME_DATE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MATCHUP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MIN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG_PCT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3M', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3A', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key=

In [14]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
def get_compiled_model():
  model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [15]:
test_sizes = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
batch_sizes = [16, 32, 64, 128]
test_size_best = 0.05
precision_recall_accuracy_sum_best = 0
loss_best = 1
batch_size_best = batch_sizes[0]

In [16]:
for size in test_sizes:
    train, test = train_test_split(games, test_size= size)
    train, val = train_test_split(train, test_size= size)
    print(len(train), 'train examples')
    print(len(val), 'validation examples')
    print(len(test), 'test examples')

    #test
    for batch_size in batch_sizes:
        train_ds = df_to_dataset(train, batch_size=batch_size)
        val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
        test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
        mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

        model = get_compiled_model()
        history = model.fit(train_ds, epochs=20, callbacks=[es,mc], validation_data=val_ds)
        y_pred = model.predict(val_ds).round()
        y_act = np.asarray(val['WL'])
        y_act = y_act.astype('float64')
        y_act = np.reshape(y_act, y_pred.shape)
        precision = precision_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
        recall = recall_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
        accuracy = history.history['val_accuracy'][-1]
        loss = history.history['val_loss'][-1]
        print('Precision= ', precision)
        print('Recall= ', recall)
        print('Accuracy= ', accuracy)
        print('Loss= ', loss)
        if precision + recall + accuracy > precision_recall_accuracy_sum_best and loss < loss_best:
            test_size_best = size
            precision_recall_accuracy_sum_best = precision + recall + accuracy
            loss_best = loss
            batch_size_best = batch_size

24296 train examples
2700 validation examples
3000 test examples
Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1499/1519 [============================>.] - ETA: 0s - loss: 0.5518 - accuracy: 0.8983
Epoch 00001: val_accuracy improved from -inf to 0.97778, saving model to best_model.hdf5
1519/1519 [==============================] - 3s 2ms/step - loss: 0.5519 - accuracy: 0.8987 - val_loss: 0.5177 - val_accuracy: 0.9778
Epoch 2/20
1508/1519 [============================>.] - ETA: 0s - loss: 0.5341 - accuracy: 0.9309
Epoch 00002: val_accuracy

Epoch 6/20
753/760 [============================>.] - ETA: 0s - loss: 0.5214 - accuracy: 0.9599
Epoch 00006: val_accuracy improved from 0.99111 to 0.99481, saving model to best_model.hdf5
760/760 [==============================] - 1s 2ms/step - loss: 0.5214 - accuracy: 0.9597 - val_loss: 0.5106 - val_accuracy: 0.9948
Epoch 7/20
745/760 [============================>.] - ETA: 0s - loss: 0.5214 - accuracy: 0.9598
Epoch 00007: val_accuracy did not improve from 0.99481
760/760 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.9592 - val_loss: 0.5118 - val_accuracy: 0.9885
Epoch 8/20
739/760 [============================>.] - ETA: 0s - loss: 0.5210 - accuracy: 0.9592
Epoch 00008: val_accuracy improved from 0.99481 to 0.99593, saving model to best_model.hdf5
760/760 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.9591 - val_loss: 0.5098 - val_accuracy: 0.9959
Epoch 9/20
746/760 [============================>.] - ETA: 0s - loss: 0.5201 - a

Epoch 14/20
366/380 [===========================>..] - ETA: 0s - loss: 0.5134 - accuracy: 0.9755
Epoch 00014: val_accuracy did not improve from 0.99815
380/380 [==============================] - 1s 2ms/step - loss: 0.5139 - accuracy: 0.9751 - val_loss: 0.5095 - val_accuracy: 0.9978
Epoch 15/20
376/380 [============================>.] - ETA: 0s - loss: 0.5138 - accuracy: 0.9753
Epoch 00015: val_accuracy improved from 0.99815 to 0.99963, saving model to best_model.hdf5
380/380 [==============================] - 1s 2ms/step - loss: 0.5136 - accuracy: 0.9755 - val_loss: 0.5091 - val_accuracy: 0.9996
Epoch 16/20
358/380 [===========================>..] - ETA: 0s - loss: 0.5136 - accuracy: 0.9769
Epoch 00016: val_accuracy did not improve from 0.99963
380/380 [==============================] - 1s 2ms/step - loss: 0.5135 - accuracy: 0.9765 - val_loss: 0.5114 - val_accuracy: 0.9896
Epoch 17/20
380/380 [==============================] - ETA: 0s - loss: 0.5153 - accuracy: 0.9723
Epoch 00017: val_

1345/1355 [============================>.] - ETA: 0s - loss: 0.5593 - accuracy: 0.8950
Epoch 00001: val_accuracy improved from -inf to 0.98170, saving model to best_model.hdf5
1355/1355 [==============================] - 3s 2ms/step - loss: 0.5591 - accuracy: 0.8952 - val_loss: 0.5103 - val_accuracy: 0.9817
Epoch 2/20
1344/1355 [============================>.] - ETA: 0s - loss: 0.5372 - accuracy: 0.9273
Epoch 00002: val_accuracy did not improve from 0.98170
1355/1355 [==============================] - 2s 2ms/step - loss: 0.5372 - accuracy: 0.9275 - val_loss: 0.5084 - val_accuracy: 0.9817
Epoch 3/20
1318/1355 [============================>.] - ETA: 0s - loss: 0.5358 - accuracy: 0.9301
Epoch 00003: val_accuracy improved from 0.98170 to 0.98405, saving model to best_model.hdf5
1355/1355 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.9303 - val_loss: 0.5066 - val_accuracy: 0.9841
Epoch 4/20
1344/1355 [============================>.] - ETA: 0s - loss: 0.5351 - a

666/678 [============================>.] - ETA: 0s - loss: 0.5209 - accuracy: 0.9626
Epoch 00008: val_accuracy did not improve from 0.99085
678/678 [==============================] - 1s 2ms/step - loss: 0.5206 - accuracy: 0.9628 - val_loss: 0.5048 - val_accuracy: 0.9859
Epoch 9/20
671/678 [============================>.] - ETA: 0s - loss: 0.5224 - accuracy: 0.9584
Epoch 00009: val_accuracy improved from 0.99085 to 0.99320, saving model to best_model.hdf5
678/678 [==============================] - 1s 2ms/step - loss: 0.5224 - accuracy: 0.9585 - val_loss: 0.5031 - val_accuracy: 0.9932
Epoch 10/20
657/678 [============================>.] - ETA: 0s - loss: 0.5225 - accuracy: 0.9597
Epoch 00010: val_accuracy did not improve from 0.99320
678/678 [==============================] - 1s 2ms/step - loss: 0.5226 - accuracy: 0.9597 - val_loss: 0.5040 - val_accuracy: 0.9885
Epoch 11/20
678/678 [==============================] - ETA: 0s - loss: 0.5216 - accuracy: 0.9606
Epoch 00011: val_accuracy did 

...

95/95 [==============================] - 0s 4ms/step - loss: 0.5325 - accuracy: 0.9465 - val_loss: 0.5278 - val_accuracy: 0.9675
Epoch 4/20
72/95 [=====================>........] - ETA: 0s - loss: 0.5282 - accuracy: 0.9568
Epoch 00004: val_accuracy improved from 0.96754 to 0.97737, saving model to best_model.hdf5
95/95 [==============================] - 0s 4ms/step - loss: 0.5255 - accuracy: 0.9579 - val_loss: 0.5183 - val_accuracy: 0.9774
Epoch 5/20
68/95 [====================>.........] - ETA: 0s - loss: 0.5192 - accuracy: 0.9688
Epoch 00005: val_accuracy improved from 0.97737 to 0.98249, saving model to best_model.hdf5
95/95 [==============================] - 0s 4ms/step - loss: 0.5204 - accuracy: 0.9690 - val_loss: 0.5137 - val_accuracy: 0.9825
Epoch 6/20
73/95 [======================>.......] - ETA: 0s - loss: 0.5182 - accuracy: 0.9730
Epoch 00006: val_accuracy improved from 0.98249 to 0.98949, saving model to best_model.hdf5
95/95 [==============================] - 0s 3ms/step - 

22/48 [============>.................] - ETA: 0s - loss: 0.5128 - accuracy: 0.9812
Epoch 00010: val_accuracy improved from 0.98060 to 0.98303, saving model to best_model.hdf5
48/48 [==============================] - 0s 5ms/step - loss: 0.5143 - accuracy: 0.9832 - val_loss: 0.5140 - val_accuracy: 0.9830
Epoch 11/20
22/48 [============>.................] - ETA: 0s - loss: 0.5137 - accuracy: 0.9844
Epoch 00011: val_accuracy improved from 0.98303 to 0.98734, saving model to best_model.hdf5
48/48 [==============================] - 0s 5ms/step - loss: 0.5131 - accuracy: 0.9840 - val_loss: 0.5117 - val_accuracy: 0.9873
Epoch 12/20
27/48 [===============>..............] - ETA: 0s - loss: 0.5149 - accuracy: 0.9829
Epoch 00012: val_accuracy improved from 0.98734 to 0.99124, saving model to best_model.hdf5
48/48 [==============================] - 0s 5ms/step - loss: 0.5132 - accuracy: 0.9824 - val_loss: 0.5108 - val_accuracy: 0.9912
Epoch 13/20
27/48 [===============>..............] - ETA: 0s - l

268/300 [=========================>....] - ETA: 0s - loss: 0.5260 - accuracy: 0.9433
Epoch 00017: val_accuracy did not improve from 0.98611
300/300 [==============================] - 1s 3ms/step - loss: 0.5268 - accuracy: 0.9429 - val_loss: 0.5130 - val_accuracy: 0.9808
Epoch 18/20
279/300 [==========================>...] - ETA: 0s - loss: 0.5312 - accuracy: 0.9303
Epoch 00018: val_accuracy did not improve from 0.98611
300/300 [==============================] - 1s 3ms/step - loss: 0.5299 - accuracy: 0.9331 - val_loss: 0.5117 - val_accuracy: 0.9857
Epoch 19/20
271/300 [==========================>...] - ETA: 0s - loss: 0.5323 - accuracy: 0.9299
Epoch 00019: val_accuracy did not improve from 0.98611
300/300 [==============================] - 1s 3ms/step - loss: 0.5330 - accuracy: 0.9292 - val_loss: 0.5143 - val_accuracy: 0.9765
Epoch 20/20
277/300 [==========================>...] - ETA: 0s - loss: 0.5329 - accuracy: 0.9273
Epoch 00020: val_accuracy did not improve from 0.98611
300/300 [==

Epoch 4/20
65/75 [=========================>....] - ETA: 0s - loss: 0.5335 - accuracy: 0.9428
Epoch 00004: val_accuracy improved from 0.91527 to 0.95430, saving model to best_model.hdf5
75/75 [==============================] - 0s 4ms/step - loss: 0.5336 - accuracy: 0.9439 - val_loss: 0.5391 - val_accuracy: 0.9543
Epoch 5/20
67/75 [=========================>....] - ETA: 0s - loss: 0.5247 - accuracy: 0.9611
Epoch 00005: val_accuracy improved from 0.95430 to 0.96833, saving model to best_model.hdf5
75/75 [==============================] - 0s 4ms/step - loss: 0.5266 - accuracy: 0.9600 - val_loss: 0.5266 - val_accuracy: 0.9683
Epoch 6/20
67/75 [=========================>....] - ETA: 0s - loss: 0.5226 - accuracy: 0.9615
Epoch 00006: val_accuracy improved from 0.96833 to 0.97666, saving model to best_model.hdf5
75/75 [==============================] - 0s 4ms/step - loss: 0.5228 - accuracy: 0.9625 - val_loss: 0.5220 - val_accuracy: 0.9767
Epoch 7/20
72/75 [===========================>..] - ETA

Epoch 11/20
27/38 [====================>.........] - ETA: 0s - loss: 0.5173 - accuracy: 0.9766
Epoch 00011: val_accuracy improved from 0.97764 to 0.98125, saving model to best_model.hdf5
38/38 [==============================] - 0s 5ms/step - loss: 0.5169 - accuracy: 0.9731 - val_loss: 0.5188 - val_accuracy: 0.9812
Epoch 12/20
24/38 [=================>............] - ETA: 0s - loss: 0.5192 - accuracy: 0.9746
Epoch 00012: val_accuracy improved from 0.98125 to 0.98375, saving model to best_model.hdf5
38/38 [==============================] - 0s 6ms/step - loss: 0.5154 - accuracy: 0.9756 - val_loss: 0.5178 - val_accuracy: 0.9837
Epoch 13/20
28/38 [=====================>........] - ETA: 0s - loss: 0.5124 - accuracy: 0.9841
Epoch 00013: val_accuracy improved from 0.98375 to 0.98611, saving model to best_model.hdf5
38/38 [==============================] - 0s 5ms/step - loss: 0.5145 - accuracy: 0.9829 - val_loss: 0.5162 - val_accuracy: 0.9861
Epoch 14/20
27/38 [====================>.........] -

In [ ]:
print(test_size_best)
print(batch_size_best)

In [ ]:
train, test = train_test_split(games, test_size= test_size_best)     #0.2
train, val = train_test_split(train, test_size= test_size_best)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


In [ ]:
test

In [ ]:
batch_size = batch_size_best    #128
train_ds = df_to_dataset(train, batch_size=batch_size)    
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

epochs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
epoch_best = epochs[0]
precision_recall_accuracy_sum_best = 0
loss_best = 1
for epoch in epochs:
    model = get_compiled_model()
    history = model.fit(train_ds, epochs=epoch, callbacks=[es,mc], validation_data=val_ds)
    y_pred = model.predict(val_ds).round()
    y_act = np.asarray(val['WL'])
    y_act = y_act.astype('float64')
    y_act = np.reshape(y_act, y_pred.shape)
    precision = precision_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
    recall = recall_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
    accuracy = history.history['val_accuracy'][-1]
    loss = history.history['val_loss'][-1]
    print('Precision= ', precision)
    print('Recall= ', recall)
    print('Accuracy= ', accuracy)
    print('Loss= ', loss)
    if precision + recall + accuracy > precision_recall_accuracy_sum_best and loss < loss_best:
        epoch_best = epoch

In [ ]:
print(epoch_best)

In [ ]:
model = get_compiled_model()
history = model.fit(train_ds, epochs=epoch_best, callbacks=[es,mc], validation_data=val_ds)
y_pred = model.predict(val_ds).round()
y_act = np.asarray(val['WL'])
y_act = y_act.astype('float64')
y_act = np.reshape(y_act, y_pred.shape)
precision = precision_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
recall = recall_score(y_act, y_pred , average="macro", labels=np.unique(y_pred))
accuracy = history.history['val_accuracy'][-1]
loss = history.history['val_loss'][-1]
print('Precision= ', precision)
print('Recall= ', recall)
print('Accuracy= ', accuracy)
print('Loss= ', loss)

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
y_pred = model.predict(test_ds)
for i, j in zip(y_pred, test['WL']):
    print('predicted: ', i, 'actual: ', j)